# Crash Course in Neural Networks with Keras
This is a brief introduction to using Keras, which will also teach you a bit about neural networks along the way.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import keras, glob
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


## 0. Neural Network = Model
In broad terms machine learning is teaching a machine the relation between some input, $x$ and some output $y$, so 

$y = f(x, a) $

where $a$ is a set of parameters that define the function $f$.

A neural network can therefore be thought of as a model $f$ that has a set of parameters, $a$, that you try to optimize to best predict the relation between, a set of training images and their labels. 

Neural networks can also be used for regression problems, and very simplified analogy would be a very complicated, high-order polynomial. 

### Sequential models
Most neural networks, or models, you can construct with Keras fall under the sequential model type. This is the 'typical' network seen in most illustrations, and consists of a sequence of layers that each take inputs from neurons in the preceding layer, perform some operation depending on the layer type, and then feed it forward into the next layer in the sequence.

In 'classical' code, this would look something like:
```python
y = input(x)
y = layer1_func(y)
y = layer2_func(y)
y = output(y)
```

Keras gives access to many types of layers, corresponding to different arithmetic functions and array manipulations. Keras allows you to mix and match these layers in your sequence, and will automatically sort out the connections between them. 

In [2]:
# Initialising a sequential model
model = Sequential()

## 1. Classification
This sets up a simple image classifier, to classify images of cats and dogs.

### Setting up data
#### Training data
To set up a training set, all you need to do it create a directory, where sub-directories correspond to each class. 

Example:

/home/training_set/cats

/home/training_set/dogs

Keras then only needs to be pointed at /home/training_set and it will figure out the classes, in this case cats and dogs.

#### Validation data
The training data that the network never sees.

#### Testing data
The data neither you or the network have seen. Performance on this is what you put in the paper.

In [3]:
# Directory where you have placed your training data
training_dir = '/home/nielsemb/work/Teaching/Keras_Tutorial/training_set' # EDIT THIS
validation_dir = '/home/nielsemb/work/Teaching/Keras_Tutorial/validation_set' # EDIT THIS
testing_dir = '/home/nielsemb/work/Teaching/Keras_Tutorial/test_set' # EDIT THIS

### Data Generators
Data generators are functions used for ingesting data and feeding it into your network, with some degree of manupulations along the way. They are typically used to feed in small batches of the training data, so as to conserve memory in case the training set is very large.

For image classification Keras has a built in class ImageDataGenerator that does this, and also preprocesses the images, and can perform data augmentation. 

#### Preprocessing
Image data comes in a variety of different shapes, sizes and formats. Your network will only function or at least function well, on images that are preprocessed in a similar fashion to what it's been trained on. It's therefore useful to establish a format specific for your network. These include, rescaling the pixel values to be between 0 and 255 (png standard), binning or interpolating the image to a specific shape. 

#### Data Augmentation
In general purpose classification it is sometimes useful to alter the training data in some way. This makes the network able to handle a greater variation in the unseen data. It can also be used to artificially increase the number of training examples.

In [4]:
# Image preprocessing
image_dim = (64, 64) # Image dimensions, important for network arch.
rescale = 1./255 # Rescale pixel values to minimum 0, maximum 255

# Data Augmentation
shear_range = 0.2 # Randomly shearing the image alonge either axis
zoom_range = 0.2 # Randomly zoom in on parts of the image 
horizontal_flip = True # Randomly flip images horizontaly

# Training-time parameters
batchsize = 32 # no. of images generator yields each call
shuffle = True # randomize the training set
class_mode = 'binary' # specific to network type, classifiers can be 'binary' or 'categorical'

# Initialize the ImageDataGenerator class
train_datagen = ImageDataGenerator(rescale = rescale, 
                                   shear_range = shear_range, 
                                   zoom_range = zoom_range,
                                   horizontal_flip = horizontal_flip) 

# Setup the ImageDataGenerator instance to ingest images from training_dir
training_set = train_datagen.flow_from_directory(training_dir,
                                                 target_size = image_dim, 
                                                 batch_size = batchsize, 
                                                 shuffle = shuffle,
                                                 class_mode = class_mode)


# Initialize the ImageDataGenerator class
validation_datagen = ImageDataGenerator(rescale = 1./255)

# Setup the ImageDataGenerator instance to ingest images from training_dir
validation_set = validation_datagen.flow_from_directory(validation_dir,
                                                        target_size = image_dim,
                                                        batch_size = batchsize,
                                                        class_mode = class_mode)

Found 2536 images belonging to 2 classes.
Found 799 images belonging to 2 classes.


### Building the network

In [5]:
conv_kernel = (3,3)
model.add(Conv2D(32, conv_kernel, input_shape = (image_dim[0], image_dim[1], 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.1))
model.add(Flatten())
model.add(Dense(units = 1, activation = 'sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the network

In [8]:
model.fit_generator(training_set, 
                    steps_per_epoch = int(training_set.samples/batchsize), 
                    epochs = 25, 
                    validation_data = validation_set,
                    validation_steps = int(validation_set.samples/batchsize))

Instructions for updating:
Use tf.cast instead.
Epoch 1/25
79/79 [==============================] - 6s 74ms/step - loss: 0.6845 - acc: 0.5672 - val_loss: 0.7402 - val_acc: 0.5026
Epoch 2/25
79/79 [==============================] - 5s 66ms/step - loss: 0.6288 - acc: 0.6448 - val_loss: 0.6604 - val_acc: 0.6076
Epoch 3/25
79/79 [==============================] - 5s 66ms/step - loss: 0.6082 - acc: 0.6642 - val_loss: 0.6231 - val_acc: 0.6610
Epoch 4/25
79/79 [==============================] - 5s 66ms/step - loss: 0.5790 - acc: 0.6867 - val_loss: 0.6604 - val_acc: 0.6310
Epoch 5/25
79/79 [==============================] - 5s 66ms/step - loss: 0.5727 - acc: 0.7029 - val_loss: 0.7624 - val_acc: 0.5724
Epoch 6/25
79/79 [==============================] - 5s 66ms/step - loss: 0.5621 - acc: 0.7065 - val_loss: 0.6003 - val_acc: 0.6832
Epoch 7/25
79/79 [==============================] - 5s 66ms/step - loss: 0.5485 - acc: 0.7156 - val_loss: 0.5979 - val_acc: 0.6845
Epoch 8/25
79/79 [=================

### Testing the network

In [17]:
from keras.preprocessing import image

# Part 3 - Making new predictions
random_image_path = np.random.choice(glob.glob(testing_dir+'/*/*'))

test_image = image.load_img(random_image_path, target_size = image_dim)
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

print(random_image_path)    
print(prediction)

/home/nielsemb/work/Teaching/Keras_Tutorial/test_set/dogs/dog.4027.jpg
dog


## 2. Regression

## 3. Transfer Learning